In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
wordsforthewise_lending_club_path = kagglehub.dataset_download('wordsforthewise/lending-club')

print('Data source import complete.')


Data source import complete.


# **INSTALL PYSPARK**

In [2]:
!pip install pyspark

## Import Libraries

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import logging
import sys
import os
import plotly.express as px
import plotly.graph_objects as go

print('Import Libraries')

Import Libraries


In [4]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

## Create Spark Session

In [5]:
spark = SparkSession.builder                                        \
                    .appName("LendingClubAnalysis")                 \
                    .config("spark.ui.showConsoleProgress","false") \
                    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

## Read Data

In [6]:
accepted_df = spark.read.option("header", True)                                             \
                        .option("inferSchema", True)                                        \
                        .option("multiLine", True)                                          \
                        .option("escape", '"')                                              \
                        .csv("/kaggle/input/lending-club/accepted_2007_to_2018Q4.csv.gz")

rejected_df = spark.read.option("header", True)                                             \
                        .option("inferSchema", True)                                        \
                        .option("multiLine", True)                                          \
                        .option("escape", '"')                                              \
                        .csv("/kaggle/input/lending-club/rejected_2007_to_2018Q4.csv.gz")

In [7]:
accepted_df.write.mode("overwrite").parquet("/kaggle/working/accepted_df.parquet")
rejected_df.write.mode("overwrite").parquet("/kaggle/working/rejected_df.parquet")

In [8]:
accepted_df = spark.read.parquet("/kaggle/working/accepted_df.parquet")
rejected_df = spark.read.parquet("/kaggle/working/rejected_df.parquet")

# **EDA**

## Total Records

In [9]:
print("Accepted Count:", accepted_df.count())
print("Rejected Count:", rejected_df.count())

Accepted Count: 2260701
Rejected Count: 27648741


## Loan Status

In [10]:
accepted_df.groupBy('loan_status').count().toPandas()

,loan_status,count
0,Fully Paid,1076751
1,Default,40
2,None,33
3,In Grace Period,8436
4,Charged Off,268559
5,Late (31-120 days),21467
6,Current,878317
7,Late (16-30 days),4349
8,Does not meet the credit policy. Status:Fully Paid,1988
9,Does not meet the credit policy. Status:Charged Off,761


## Bad Loan Rates
* Charge-Off
* Default
* Delinquency
* In Grace Period

In [11]:
from pyspark.sql.functions import round as spark_round

# Define 'bad_loans'
df = accepted_df.withColumn('bad_loans',
                            when(col('loan_status').isin(['Charged Off', 'Does not meet the credit policy. Status:Charged Off']), 'Charge-Off Rate')
                            .when(col('loan_status') == 'Default', 'Default Rate')
                            .when(col('loan_status').isin(['Late (16-30 days)', 'Late (31-120 days)']), 'Delinquency Rate')
                            .when(col('loan_status') == 'In Grace Period', 'In Grace Rate')
                           )

# Total Loans
total_loan_count = accepted_df.count()

# Percentages
summary = df.groupBy('bad_loans') \
            .agg(count('*').alias('count')) \
            .filter(col('bad_loans').isNotNull()) \
            .withColumn('percentage', spark_round(col('count') / total_loan_count * 100, 2)) \
            .toPandas()
summary

,bad_loans,count,percentage
0,In Grace Rate,8436,0.37
1,Delinquency Rate,25816,1.14
2,Charge-Off Rate,269320,11.91
3,Default Rate,40,0.00


## Timeframe
* rejected_df - Application Date
* accepted_df - Issue Date

In [12]:
rejected_df.agg(
                min(col('Application Date')).alias('Earliest Application Date'),
                max(col('Application Date')).alias('Latest Application Date')
                )\
           .toPandas()

,Earliest Application Date,Latest Application Date
0,2007-05-26,2018-12-31


In [13]:
# 'issue_d' was stored in string format --> need to cast to date format

accepted_df.withColumn('issue_date', to_date(col('issue_d'), 'MMM-yyyy')) \
           .select(
                date_format(min('issue_date'), 'MMM-yyyy').alias('Earliest Issue Date'),
                date_format(max('issue_date'), 'MMM-yyyy').alias('Latest Issue Date')
            ).toPandas()

,Earliest Issue Date,Latest Issue Date
0,Jun-2007,Dec-2018


## Grade, Sub_grade

In [14]:
# Subgrade
accepted_df.groupBy("grade", "sub_grade") \
    .count() \
    .orderBy('grade', 'sub_grade') \
    .toPandas()

,grade,sub_grade,count
0,None,None,33
1,A,A1,86790
2,A,A2,69562
3,A,A3,73184
4,A,A4,95874
5,A,A5,107617
6,B,B1,125341
7,B,B2,126621
8,B,B3,131514
9,B,B4,139793


## Duplicates check

In [15]:
# Total distinct id count
print("Unique IDs:", accepted_df.select('id').distinct().count())

# Total duplicate id count
print("Duplicates:", accepted_df.groupBy('id').count().filter(col('count')>1).count())

Unique IDs: 2260701
Duplicates: 0


# **ANALYSIS**

## 1. Loan Issuance

### a. Overtime

In [16]:
# Loan Issuance Overtime

issuance_vintage = accepted_df.withColumn('issue_date', to_date(col('issue_d'), 'MMM-yyyy')) \
                              .select(
                                    year(col('issue_date')).alias('Year')
                                  ) \
                              .groupBy('Year') \
                              .count() \
                              .orderBy('Year') \
                              .toPandas()

issuance_vintage = issuance_vintage.dropna(subset=['Year'])

issuance_vintage

,Year,count
1,2007.0,603
2,2008.0,2393
3,2009.0,5281
4,2010.0,12537
5,2011.0,21721
6,2012.0,53367
7,2013.0,134814
8,2014.0,235629
9,2015.0,421095
10,2016.0,434407


In [17]:
# Data Visualization

issuance_sorted = issuance_vintage.sort_values(by=['Year'])

# plot
fig = px.line(
    issuance_sorted,
    x='Year',
    y='count',
    title='Loan Issuance Overtime',
    markers=True
)

fig.update_layout(
    xaxis_title= '',
    yaxis_title= '',
    template= 'simple_white'
)

fig.show()

### b. By Vintages

In [18]:
# Loan Issuance grouped by Vintages

issuance = accepted_df.withColumn('issue_date', to_date(col('issue_d'), 'MMM-yyyy')) \
                      .select(
                            date_format(col('issue_date'), 'MMM').alias('Month'),
                            year(col('issue_date')).alias('Year')
                             ) \
                      .groupBy('Year', 'Month') \
                      .count() \
                      .orderBy('Year', 'Month') \
                      .dropna(subset=['Year', 'Month']) \
                      .toPandas()

issuance

,Year,Month,count
0,2007,Aug,74
1,2007,Dec,172
2,2007,Jul,63
3,2007,Jun,24
4,2007,Nov,112
...,...,...,...
134,2018,Mar,38771
135,2018,May,46311
136,2018,Nov,41973
137,2018,Oct,46305


In [19]:
# Data Visualization

month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
issuance['Month'] = pd.Categorical(issuance['Month'], categories=month_order, ordered=True)
issuance_sorted = issuance.sort_values(by=['Year', 'Month'])

# plot
fig = px.line(
    issuance_sorted,
    x='Month',
    y='count',
    color='Year',
    title='Loan Issuance By Vintages',
    markers=True,
    category_orders={'Month': month_order}
)

fig.update_layout(
    xaxis_title='',
    yaxis_title='',
    legend_title='Year',
    template='simple_white'
)

fig.show()

## 2. Loan Distribution

### a. Loan Quality Distribution by Grade (Volume)

In [20]:
# Define 'loan_quality'
accepted_df = accepted_df.dropna(subset=['grade']) \
                         .withColumn('loan_quality',
                                     when(col('loan_status').isin("Charged Off", "Default", "Does not meet the credit policy. Status:Charged Off",
                                                                  "Late (16-30 days)", "Late (31-120 days)", "In Grace Period"), "Bad Loan")
                                     .otherwise("Good Loan"))

# Total loans by 'grade' and 'loan_quality'
loan_by_grade = accepted_df.groupBy('grade', 'loan_quality') \
                           .agg(count('*').alias('count'))

# Total loans by 'grade'
grade_totals = loan_by_grade.groupBy('grade') \
                            .agg(sum('count').alias('total_count'))

# Percentage
loan_by_grade = loan_by_grade.join(grade_totals, on='grade') \
                             .withColumn('percentage', round(col('count') / col('total_count') * 100, 2)) \
                             .select('grade', 'loan_quality', 'count', 'total_count', 'percentage')

loan_by_grade = loan_by_grade.orderBy('grade', 'loan_quality').toPandas()
loan_by_grade

,grade,loan_quality,count,total_count,percentage
0,A,Bad Loan,16509,433027,3.81
1,A,Good Loan,416518,433027,96.19
2,B,Bad Loan,60213,663557,9.07
3,B,Good Loan,603344,663557,90.93
4,C,Bad Loan,97828,650053,15.05
5,C,Good Loan,552225,650053,84.95
6,D,Bad Loan,68893,324424,21.24
7,D,Good Loan,255531,324424,78.76
8,E,Bad Loan,39566,135639,29.17
9,E,Good Loan,96073,135639,70.83


In [21]:
# Data Visualization

loan_by_grade['loan_quality'] = pd.Categorical(loan_by_grade['loan_quality'], categories=['Bad Loan', 'Good Loan'], ordered=True)

loan_by_grade_sorted = loan_by_grade.sort_values(['grade', 'loan_quality'])

fig = px.bar(
    loan_by_grade_sorted,
    x='grade',
    y='count',
    color='loan_quality',
    barmode='group',
    title='Loan Quality Distribution by Grade (Volume)'
)

fig.update_layout(
    xaxis_title='',
    yaxis_title='',
    legend_title='Loan Quality',
    template='simple_white'
)

fig.show()

### b. Loan Quality Distribution by Grade (Percentage)

In [22]:
# Data Visualization

loan_by_grade['loan_quality'] = pd.Categorical(loan_by_grade['loan_quality'], categories=['Bad Loan', 'Good Loan'], ordered=True)

loan_by_grade_sorted = loan_by_grade.sort_values(['grade', 'loan_quality'])

fig = px.bar(
    loan_by_grade_sorted,
    x='grade',
    y='percentage',
    color='loan_quality',
    barmode='group',
    title='Loan Quality Distribution by Grade (Percentage)'
)

fig.update_layout(
    xaxis_title='',
    yaxis_title='',
    legend_title='Loan Quality',
    template='simple_white'
)

fig.show()

### c. Loan Status Distribution by Grade (Volume)

In [23]:
# 'loan_status' grouping
accepted_df = accepted_df.dropna(subset=['grade']) \
                         .withColumn('loan_quality',
                                     when(col('loan_status').isin("Charged Off", "Default", "Does not meet the credit policy. Status:Charged Off",
                                                                  "Late (16-30 days)", "Late (31-120 days)", "In Grace Period"),"Bad loan")
                                     .otherwise("Good loan")
                                    ) \
                         .withColumn('loan_status_grouped',
                                     when(col('loan_status').isin("Charged Off", "Does not meet the credit policy. Status:Charged Off"), "Charge Off")
                                     .when(col('loan_status') == "Default", "Default")
                                     .when(col('loan_status').isin("Late (16-30 days)", "Late (31-120 days)"), "Delinquency")
                                     .when(col('loan_status') == "In Grace Period", "In Grace")
                                     .otherwise("Good loan"))

# Total loans by 'loan_status' in each 'grade' & 'loan_quality'
loan_by_status = accepted_df.groupBy('grade', 'loan_quality', 'loan_status_grouped') \
                            .agg(count('*').alias('count'))

# Total loans by 'grade'
grade_totals = loan_by_status.groupBy('grade') \
                             .agg(sum('count').alias('total_count'))

# Percentage
loan_by_status = loan_by_status.join(grade_totals, on='grade') \
                               .withColumn('percentage', round(col('count') / col('total_count') * 100, 2)) \
                               .orderBy('grade', 'loan_quality', 'loan_status_grouped') \
                               .toPandas()

loan_by_status

,grade,loan_quality,loan_status_grouped,count,total_count,percentage
0,A,Bad loan,Charge Off,14209,433027,3.28
1,A,Bad loan,Default,5,433027,0.00
2,A,Bad loan,Delinquency,1663,433027,0.38
3,A,Bad loan,In Grace,632,433027,0.15
4,A,Good loan,Good loan,416518,433027,96.19
5,B,Bad loan,Charge Off,52654,663557,7.94
6,B,Bad loan,Default,7,663557,0.00
7,B,Bad loan,Delinquency,5781,663557,0.87
8,B,Bad loan,In Grace,1771,663557,0.27
9,B,Good loan,Good loan,603344,663557,90.93


In [24]:
filtered_df = loan_by_status[loan_by_status['loan_status_grouped'] != 'Good loan']

fig = px.bar(
    filtered_df,
    x='grade',
    y='count',
    color='loan_status_grouped',
    barmode='group',
    title='Bad Loan Status Distribution by Grade (Volume)'
)

fig.update_layout(
    xaxis_title='',
    yaxis_title='',
    legend_title='Loan Status',
    template='simple_white'
)

fig.show()

In [25]:
filtered_df = loan_by_status[loan_by_status['loan_status_grouped'] != 'Good loan']

fig = px.bar(
    filtered_df,
    x='grade',
    y='percentage',
    color='loan_status_grouped',
    barmode='group',
    title='Bad Loan Status Distribution by Grade (Percentage)'
)

fig.update_layout(
    xaxis_title='',
    yaxis_title='',
    legend_title='Loan Status',
    template='simple_white'
)

fig.show()

## 3. Charge-Off

### a. Charge-Off Overtime

In [26]:
#Charge-Off Overtime

chargeoff = accepted_df.withColumn('issue_date', to_date(col('issue_d'), 'MMM-yyyy')) \
                       .select(
                                year(col('issue_date')).alias('Year')
                              ) \
                       .filter(col('loan_status').isin(['Charged Off', 'Does not meet the credit policy. Status:Charged Off'])) \
                       .groupBy('Year') \
                       .count() \
                       .orderBy('Year') \
                       .dropna(subset=['Year']) \
                       .toPandas()
chargeoff

,Year,count
0,2007,158
1,2008,496
2,2009,723
3,2010,1757
4,2011,3297
5,2012,8644
6,2013,21024
7,2014,41161
8,2015,75803
9,2016,68242


In [27]:
# Data Visualization

chargeoff = chargeoff.sort_values(by=['Year'])

# plot
fig = px.line(
    chargeoff,
    x='Year',
    y='count',
    title='Charge-Off Overtime',
    markers=True
)

fig.update_layout(
    xaxis_title='',
    yaxis_title='',
    template='simple_white'
)

fig.show()

## 4. Acceptance Rate

### a. Combined datasets

In [28]:
# Accpeted
accepted_c = accepted_df.withColumn('accepted_flag', lit(1)) \
                        .select('loan_amnt', 'emp_length', 'dti', 'addr_state', 'policy_code', 'accepted_flag')

# Rejected
rejected_c = rejected_df.withColumnRenamed('Amount Requested', 'loan_amnt') \
                        .withColumnRenamed('Employment Length', 'emp_length') \
                        .withColumnRenamed('State', 'addr_state') \
                        .withColumnRenamed('Policy Code', 'policy_code') \
                        .withColumn('dti', regexp_replace(col('Debt-To-Income Ratio'), '%', '').cast('double')) \
                        .withColumn('accepted_flag', lit(0)) \
                        .select('loan_amnt', 'emp_length', 'dti', 'addr_state', 'policy_code', 'accepted_flag')

# Combine 2 datsets
combined_df = accepted_c.union(rejected_c)

combined_df = combined_df.withColumn('emp_length_years',
                       when(col('emp_length') == '< 1 year', 0)
                       .when(col('emp_length') == '1 year', 1)
                       .when(col('emp_length') == '2 years', 2)
                       .when(col('emp_length') == '3 years', 3)
                       .when(col('emp_length') == '4 years', 4)
                       .when(col('emp_length') == '5 years', 5)
                       .when(col('emp_length') == '6 years', 6)
                       .when(col('emp_length') == '7 years', 7)
                       .when(col('emp_length') == '8 years', 8)
                       .when(col('emp_length') == '9 years', 9)
                       .when(col('emp_length') == '10+ years', 10)
                       .when(col('emp_length').isNull(), -1)
                       .otherwise(-1))

### b. Acceptance Rate by 'emp_length' (years)

In [29]:
# Acceptance Rate by 'emp_length' (years)

accepted_by_emp_length = combined_df.groupBy("emp_length_years", "accepted_flag") \
                                    .count() \
                                    .groupBy("emp_length_years") \
                                    .pivot("accepted_flag") \
                                    .sum("count") \
                                    .withColumn("acceptance_rate", round((col("1") / (col("1") + col("0")))*100, 2)) \
                                    .orderBy("emp_length_years") \
                                    .toPandas()
accepted_by_emp_length

,emp_length_years,0,1,acceptance_rate
0,-1,951355,146907,13.38
1,0,22994315,189988,0.82
2,1,267840,148403,35.65
3,2,199204,203677,50.56
4,3,177344,180753,50.48
5,4,121623,136605,52.90
6,5,2279466,139698,5.77
7,6,71625,102628,58.90
8,7,55666,92695,62.48
9,8,65965,91914,58.22


In [30]:
# Data Visualization

# Label 'emp_length'
all_years = pd.DataFrame({'emp_length_years': [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'emp_length_label': ['Unknown', '<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10+']})

accepted_rate_emp_length = all_years.merge(accepted_by_emp_length, on='emp_length_years', how='left')

# Plot
fig = px.line(
    accepted_rate_emp_length,
    x='emp_length_label',
    y='acceptance_rate',
    markers=True,
    title='Acceptance Rate by Employment Length',
)

fig.update_layout(
    xaxis_title='Employment Length (Years)',
    yaxis_title='Acceptance Rate (%)',
    template='simple_white'
)

fig.show()

### Loan application and Acceptance Rate

In [31]:
all_years = pd.DataFrame({'emp_length_years': [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'emp_length_label': ['Unknown', '<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10+']})

df = all_years.merge(accepted_by_emp_length, on='emp_length_years', how='left')

# Bar chart for Loan Application (counts)
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df['emp_length_label'],
    y=df['0'],
    name='Rejected',
    marker_color='lightcoral'))

fig.add_trace(go.Bar(
    x=df['emp_length_label'],
    y=df['1'],
    name='Accepted',
    marker_color='lightblue'))

# Line chart for Acceptance Rate (secondary y-axis)
fig.add_trace(go.Scatter(
    x=df['emp_length_label'],
    y=df['acceptance_rate'],
    name='Acceptance Rate (%)',
    mode='lines+markers',
    yaxis='y2',
    line=dict(color='green', width=3)))

# Layout
fig.update_layout(
    title='Loan Applications & Acceptance Rate by Employment Length',
    xaxis=dict(title='Employment Length (Years)'),
    yaxis=dict(title='Number of Loans'),
    yaxis2=dict(title='Acceptance Rate (%)', overlaying='y', side='right', showgrid=False),
    barmode='group',
    legend_title='Loan Outcome',
    template='simple_white'
)

fig.show()